In [2]:
import pandas as pd
import numpy as np
import joblib

from rdkit import Chem
from helicity import run_helicity_predictions
from hydrophobic_moment import run_hydrophobic_moment_predictions
from mapchiral.mapchiral import encode

from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, precision_score, accuracy_score, recall_score, f1_score

#### Import data

Read files

In [3]:
activity = pd.read_csv('data/activity_clean.csv')
hemolysis = pd.read_csv('data/hemolysis_clean.csv')

#### Helicity and Hydrophobic Moment

Helicity prediction using SPIDER3. Some sequences need to be removed for the prediction. For these sequences, helicity is set to 0

In [4]:
# Remove sequences which won't work for SPIDER3 predictions
activity_helicity = activity[['sequence', 'label']]
activity_helicity = activity_helicity[activity_helicity['sequence'].str.len() > 1]
activity_helicity = activity_helicity[~activity_helicity['sequence'].str.contains('[a-z]')]
activity_helicity['helicity'] = run_helicity_predictions(activity_helicity['sequence'], 'spider3', 'dbaasp_activity') 

hemolysis_helicity = hemolysis[['sequence', 'label']]
hemolysis_helicity = hemolysis_helicity[hemolysis_helicity['sequence'].str.len() > 1]
hemolysis_helicity = hemolysis_helicity[~hemolysis_helicity['sequence'].str.contains('[a-z]')]
hemolysis_helicity['helicity'] = run_helicity_predictions(hemolysis_helicity['sequence'], 'spider3', 'dbaasp_hemolysis')

# Merge back the results into the original dataframe
activity = pd.merge(activity, activity_helicity, on='sequence', how='left')
activity['helicity'] = activity['helicity'].fillna(0)

hemolysis = pd.merge(hemolysis, hemolysis_helicity, on='sequence', how='left')
hemolysis['helicity'] = hemolysis['helicity'].fillna(0)

doing iteration 0 - SS
doing iteration 0 - ASA THETA TAU PHI PSI HSEa CN
combining both prediction files
doing iteration 1 - SS
doing iteration 1 - ASA THETA TAU PHI PSI HSEa CN
combining both prediction files
Time taken - 174 seconds


Hydrophobic moment prediction using a python script from Joao Rodrigues

In [6]:
activity['hydrophobic_moment'] = activity['sequence'].apply(run_hydrophobic_moment_predictions)
hemolysis['hydrophobic_moment'] = hemolysis['sequence'].apply(run_hydrophobic_moment_predictions)

Helicity and hydrophobic moment are combined into a single feature vector

In [7]:
def combine_features(row):
    return [row['helicity'], row['hydrophobic_moment']]

activity['features'] = activity.apply(combine_features, axis=1)
hemolysis['features'] = hemolysis.apply(combine_features, axis=1)

#### MAP4C fingerprint

Add the MAP4C fingerprint feature vector

In [12]:
activity['map4'] = activity['sequence'].apply(lambda x: encode(Chem.MolFromSequence(x), max_radius=2, n_permutations=2048)) 
hemolysis['map4'] = hemolysis['sequence'].apply(lambda x: encode(Chem.MolFromSequence(x), max_radius=2, n_permutations=2048))

#### Export data frames and load exported data frames

Export data frames using the joblib library

In [20]:
joblib.dump(activity, 'data/activity_clean_features.pkl')
joblib.dump(hemolysis, 'data/hemolysis_clean_features.pkl')

['data_clean/hemolysis_clean_features.pkl']

Read data frames using the joblib library

In [4]:
activity = joblib.load('data/activity_clean_features.pkl')
hemolysis = joblib.load('data/hemolysis_clean_features.pkl')

#### SVM classifier

Define Jaccard kernel function for SVM. This kernel function is required for the MAP4C fingerprint. \
Define helper function for the evaluation of the classifier. 

In [37]:
def jaccard_kernel(X, Y=None):
    if Y is None:
        X = Y
    js_allpairs = np.zeros((len(X),len(Y)))
    for i,fp1 in enumerate(X):
        for j,fp2 in enumerate(Y):
            js_allpairs[i,j] = np.float64(np.count_nonzero(fp1 == fp2)) / np.float64(len(fp1))
    return js_allpairs


def calculate_metrics(y_true, y_pred):
    
    roc_auc = roc_auc_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    return pd.DataFrame({'roc_auc': [roc_auc], 'accuracy': [accuracy], 'precision': [precision], 'recall': [recall], 'f1': [f1]})

def run_cross_validation(df, feature, purpose):

    # Initialize the output data frame
    df_out = pd.DataFrame()
    
    # Loop over the five folds
    for i in range(1, 6):

        # Split the data based on the information in the data frame column
        X_train, y_train = df[df[f'split_{i}'] == 'training'][feature].values.tolist(), df[df[f'split_{i}'] == 'training']['label'].values.tolist()
        X_test, y_test = df[df[f'split_{i}'] == 'test'][feature].values.tolist(), df[df[f'split_{i}'] == 'test']['label'].values.tolist()

        # Train the model 
        if feature == 'map4':
            svm = SVC(C=1, kernel=jaccard_kernel, class_weight='balanced', random_state=42)
        elif feature == 'features':
            svm = SVC(C=1, kernel='linear', class_weight='balanced', random_state=42)
        else:
            continue
        svm.fit(np.array(X_train), np.array(y_train))

        # Export the model 
        joblib.dump(svm, f'models/SVM/svm_{purpose}_{feature}_fold_{i}.pkl')

        # Predict and append to the dataframe
        y_pred = svm.predict(X_test)
        row = calculate_metrics(y_test, y_pred)

        # Append the results to the output dataframe
        df_out = pd.concat([df_out, row], ignore_index=True)
    
    # Add mean and standard deviation
    df_out = pd.concat([df_out, pd.DataFrame(df_out.mean()).T], ignore_index=True)
    df_out = pd.concat([df_out, pd.DataFrame(df_out.std()).T], ignore_index=True)
    df_out.index = [f'fold_{i}' for i in range(1, 6)] + ['mean', 'std']

    return df_out

5-fold cross-validation on helicity and hydrophobic moment features

In [38]:
activity_features = run_cross_validation(activity, 'features', 'activity')
hemolysis_features = run_cross_validation(hemolysis, 'features', 'hemolysis')

5-fold cross-validation on MAP4C fingerprint

In [41]:
activity_map4c = run_cross_validation(activity, 'map4', 'activity')
hemolysis_map4c = run_cross_validation(hemolysis, 'map4', 'hemolysis')

#### Export data frames

In [42]:
activity_features.to_csv('results/svm_activity_features_5cv.csv')
hemolysis_features.to_csv('results/svm_hemolysis_features_5cv.csv')
activity_map4c.to_csv('results/svm_activity_map4c_5cv.csv')
hemolysis_map4c.to_csv('results/svm_hemolysis_map4c_5cv.csv')